In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 12, 6, Finished, Available)

# Set up data configuration

In [17]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 10, Finished, Available)

# Reading in single parquet file

In [18]:
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 11, Finished, Available)

# select subreddit topics we are going to analyze

In [19]:
topic = ["Tetris","pokemon","SuperMario","GTA","CallOfDuty","FIFA","legostarwars", "assassinscreed","thesims","FinalFantasy"]

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 12, Finished, Available)

# Reading in all of the Reddit data

In [20]:
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 13, Finished, Available)

In [21]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(topic))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 14, Finished, Available)

# select post with the subreddit topics

In [22]:
df_post = df_save = submissions_filtered.select("subreddit","selftext").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 15, Finished, Available)

# spark NLP pipeline build

In [12]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, -1, Finished, Available)

Unrecognized options: 

In [13]:
!pip install spark-nlp

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 6, Finished, Available)

In [14]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 7, Finished, Available)

In [15]:
documentAssembler = DocumentAssembler()\
    .setInputCol("selftext")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 8, Finished, Available)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


# run the pipeline

In [26]:
df_post = df_post.dropna()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 19, Finished, Available)

In [27]:
senti_model = nlpPipeline.fit(df_post)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 20, Finished, Available)

In [28]:
senti_pred = senti_model.transform(df_post)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 21, Finished, Available)

In [29]:
senti_pred.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 22, Finished, Available)

+------------+--------------------+--------------------+--------------------+--------------------+
|   subreddit|            selftext|            document| sentence_embeddings|           sentiment|
+------------+--------------------+--------------------+--------------------+--------------------+
|        FIFA|                    |[{document, 0, -1...|                  []|                  []|
|FinalFantasy|                    |[{document, 0, -1...|                  []|                  []|
|     pokemon|So several days a...|[{document, 0, 13...|[{sentence_embedd...|[{category, 0, 13...|
|        FIFA|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
|        FIFA|                    |[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
+------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [34]:
df_senti = senti_pred.select("subreddit",F.explode('sentiment.result'))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 27, Finished, Available)

In [36]:
df_senti = df_senti.withColumnRenamed("col", "sentiment")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 29, Finished, Available)

In [37]:
df_senti_count = df_senti.groupBy("subreddit","sentiment").count().toPandas()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 30, Finished, Available)

In [41]:
import os
CSV_DIR = os.path.join("Users/ky285/fall-2023-reddit-project-team-11/data", "csv")
df_senti_count.to_csv(f"{CSV_DIR}/sentiment_count_top10.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 34, Finished, Available)